In [97]:
import pandas as pd
import yaml
pd.set_option('display.max_columns',None)
from ast import literal_eval

In [9]:
with open('./virtual_experiment_linear_data.yaml', 'r') as f:
    #print(yaml.load(f))
    data=yaml.load(f)
    #df = pd.io.json.json_normalize(yaml.load(f))
ll=[]
for k,v in data.items():
    ll.append(data.get(k))
df =pd.DataFrame(ll,columns=['a','df','f'])
df

<ipython-input-9-87d072995765>:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data=yaml.load(f)


,a,df,f
0,1.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[1.0, 1.3333333333333333, 1.6666666666666665, ..."
1,3.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[3.0, 3.3333333333333335, 3.6666666666666665, ..."
2,5.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[5.0, 5.333333333333333, 5.666666666666667, 6...."
3,7.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[7.0, 7.333333333333333, 7.666666666666667, 8...."
4,9.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[9.0, 9.333333333333334, 9.666666666666666, 10..."
5,11.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[11.0, 11.333333333333334, 11.666666666666666,..."
6,13.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[13.0, 13.333333333333334, 13.666666666666666,..."
7,15.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[15.0, 15.333333333333334, 15.666666666666666,..."
8,17.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[17.0, 17.333333333333332, 17.666666666666668,..."
9,19.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]","[19.0, 19.333333333333332, 19.666666666666668,..."


# Genarate RDF triples

In [10]:
import rdfpandas.graph
import pandas as pd
import rdflib
from rdflib.namespace import RDF, XSD, NamespaceManager,SKOS
from rdflib import BNode, Literal, Namespace, Graph

In [75]:
# flash('RDF file successfully created')
from rdflib.namespace import RDFS
BMWD = Namespace('https://www.materials.fraunhofer.de/ontologies/BWMD_ontology/mid#')
UNIT = Namespace('http://www.ontologyrepository.com/CommonCoreOntologies/Mid/InformationEntityOntology')#
DIFFERENTIAL = Namespace( 'http://www.qudt.org/2.1/vocab/unit')
BS = Namespace('https://w3id.org/def/basicsemantics-owl#')

g = Graph()
g.namespace_manager = NamespaceManager(Graph())
g.namespace_manager.bind('unit', UNIT)
g.namespace_manager.bind('bs', BS)
g.namespace_manager.bind('bmwd', BMWD)
g.namespace_manager.bind('df', DIFFERENTIAL)
g.namespace_manager.bind('skos', SKOS)
g.namespace_manager.bind('rdfpandas', Namespace('http://github.com/cadmiumkitty/rdfpandas/'))


for i in range(0,df.shape[0]):
    g.add((BMWD["record"], rdflib.RDF.value, Literal(df['a'][i], datatype=XSD.float)))
    g.add((DIFFERENTIAL['differential_of'], rdflib.RDF.value, Literal(df['df'][i],datatype=XSD.list)))
    g.add((BS['method'], rdflib.RDF.value, Literal(df['f'][i],datatype=XSD.list)))
g.serialize("test.ttl", format="turtle")

# Example : push rdf triple to triplestore

In [80]:
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDFS
from SPARQLWrapper import SPARQLWrapper, JSON, POST, BASIC, DIGEST

###############################################################################
#main server : https://fuseki.materials.digital/modellierung/update
#test server: https://fuseki.materials.digital/windturbine
sparql = SPARQLWrapper("https://fuseki.materials.digital/modellierung/update")

sparql.setHTTPAuth(BASIC)
sparql.setCredentials("mechanik", "testmech21*")
sparql.setMethod(POST)
for s,p,o in g:
    data = s.n3() + " " +p.n3() + " "+ o.n3()+ " . \n"
    #print("""+ data + """)
    query = """
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX df: <http://www.qudt.org/2.1/vocab/unit> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX example: <https://example.org/Row>
    PREFIX bmwd: <https://www.materials.fraunhofer.de/ontologies/BWMD_ontology/mid#>
    INSERT DATA { 
    """+data+"""
    }"""
    sparql.setQuery(query)

    results = sparql.query()
    #print(results.response.read())
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
results

b'<html>\n<head>\n</head>\n<body>\n<h1>Success</h1>\n<p>\nUpdate succeeded\n</p>\n</body>\n</html>\n'

# Read same triples from triple store

Read value 'a' 

In [86]:
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDFS
data = []
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX df: <http://www.qudt.org/2.1/vocab/unit> 
PREFIX df: <http://www.qudt.org/2.1/vocab/unit>
PREFIX bwmd: <https://www.materials.fraunhofer.de/ontologies/BWMD_ontology/mid#>
PREFIX example: <https://example.org/Row>
SELECT * WHERE { 
?s ?p ?o . 
filter (contains(str(?s), "record"))
} Limit 1000
"""
###############################################################################

sparql = SPARQLWrapper("https://fuseki.materials.digital/modellierung/sparql")
sparql.setHTTPAuth(BASIC)
sparql.setCredentials("mechanik", "testmech21*")
sparql.setMethod(POST)

#sparql = SPARQLWrapper("https://example.org/sparql-auth")
#sparql.setHTTPAuth(DIGEST)
#sparql.setCredentials("login", "password")
#sparql.setMethod(POST)
sparql.setQuery(query)

results = sparql.query()
#print(results.response.read())
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    #print(result)
    data.append(result['o']['value'])
data

['1.0', '3.0', '5.0', '7.0', '9.0', '11.0', '13.0', '15.0', '17.0', '19.0']

Read value 'f'

In [98]:
data = []
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX df: <http://www.qudt.org/2.1/vocab/unit> 
PREFIX df: <http://www.qudt.org/2.1/vocab/unit>
PREFIX bwmd: <https://www.materials.fraunhofer.de/ontologies/BWMD_ontology/mid#>
PREFIX example: <https://example.org/Row>
SELECT * WHERE { 
?s ?p ?o . 
filter (contains(str(?s), "method"))
} Limit 1000
"""
###############################################################################

sparql = SPARQLWrapper("https://fuseki.materials.digital/modellierung/sparql")
sparql.setHTTPAuth(BASIC)
sparql.setCredentials("mechanik", "testmech21*")
sparql.setMethod(POST)

#sparql = SPARQLWrapper("https://example.org/sparql-auth")
#sparql.setHTTPAuth(DIGEST)
#sparql.setCredentials("login", "password")
#sparql.setMethod(POST)
sparql.setQuery(query)

results = sparql.query()
#print(results.response.read())
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    #print(result)
    data.append(literal_eval(result['o']['value']))
data

[[1.0,
  1.3333333333333333,
  1.6666666666666665,
  2.0,
  2.333333333333333,
  2.666666666666667,
  3.0,
  3.333333333333333,
  3.6666666666666665,
  4.0],
 [3.0,
  3.3333333333333335,
  3.6666666666666665,
  4.0,
  4.333333333333333,
  4.666666666666667,
  5.0,
  5.333333333333333,
  5.666666666666666,
  6.0],
 [5.0,
  5.333333333333333,
  5.666666666666667,
  6.0,
  6.333333333333333,
  6.666666666666667,
  7.0,
  7.333333333333333,
  7.666666666666666,
  8.0],
 [7.0,
  7.333333333333333,
  7.666666666666667,
  8.0,
  8.333333333333334,
  8.666666666666666,
  9.0,
  9.333333333333332,
  9.666666666666666,
  10.0],
 [9.0,
  9.333333333333334,
  9.666666666666666,
  10.0,
  10.333333333333334,
  10.666666666666666,
  11.0,
  11.333333333333332,
  11.666666666666666,
  12.0],
 [11.0,
  11.333333333333334,
  11.666666666666666,
  12.0,
  12.333333333333334,
  12.666666666666666,
  13.0,
  13.333333333333332,
  13.666666666666666,
  14.0],
 [13.0,
  13.333333333333334,
  13.666666666666

Read value 'df'

In [96]:
data = []
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX df: <http://www.qudt.org/2.1/vocab/unit> 
PREFIX df: <http://www.qudt.org/2.1/vocab/unit>
PREFIX bwmd: <https://www.materials.fraunhofer.de/ontologies/BWMD_ontology/mid#>
PREFIX example: <https://example.org/Row>
SELECT * WHERE { 
?s ?p ?o . 
filter (contains(str(?s), "differential_of"))
} Limit 1000
"""
###############################################################################

sparql = SPARQLWrapper("https://fuseki.materials.digital/modellierung/sparql")
sparql.setHTTPAuth(BASIC)
sparql.setCredentials("mechanik", "testmech21*")
sparql.setMethod(POST)

#sparql = SPARQLWrapper("https://example.org/sparql-auth")
#sparql.setHTTPAuth(DIGEST)
#sparql.setCredentials("login", "password")
#sparql.setMethod(POST)
sparql.setQuery(query)

results = sparql.query()
#print(results.response.read())
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    #print(result)
    data.append(literal_eval(result['o']['value']))
#literal_eval(data[0])
data

[[3.0, 3.0, 3.0, 3.0, 3.0, 3.0]]